# Sales Tax Data

In [30]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

In [31]:
blocks = pd.read_csv("raw_data/census_blocks_2020.csv", dtype={"GEOID20": "string"})
blocks.head()

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,HOUSING20,POP20,data_as_of,data_loaded_at,multipolygon
0,6,75,10101,1001,60750101011001,Block 1001,G5040,S,"262,902",0,37.808484,-122.409904,0,15,2022 Jul 21 10:57:39 PM,2022 Jul 25 01:59:00 PM,"MULTIPOLYGON (((-122.420353 37.81151, -122.420..."
1,6,75,47801,1005,60750478011005,Block 1005,G5040,S,"19,608",0,37.773038,-122.494435,71,173,2022 Jul 21 10:57:39 PM,2022 Jul 25 01:59:00 PM,"MULTIPOLYGON (((-122.495039 37.773947, -122.49..."
2,6,75,15401,1005,60750154011005,Block 1005,G5040,S,"14,331",0,37.783112,-122.448710,44,150,2022 Jul 21 10:57:39 PM,2022 Jul 25 01:59:00 PM,"MULTIPOLYGON (((-122.449358 37.783692, -122.44..."
3,6,75,32901,2003,60750329012003,Block 2003,G5040,S,"19,646",0,37.747121,-122.489407,55,168,2022 Jul 21 10:57:39 PM,2022 Jul 25 01:59:00 PM,"MULTIPOLYGON (((-122.490009 37.748031, -122.48..."
4,6,75,15401,2002,60750154012002,Block 2002,G5040,S,"26,899",0,37.782709,-122.456171,62,178,2022 Jul 21 10:57:39 PM,2022 Jul 25 01:59:00 PM,"MULTIPOLYGON (((-122.456813 37.783929, -122.45..."


In [32]:
# turn into shapely object
blocks["geometry"] = blocks["multipolygon"].apply(wkt.loads)

In [33]:
#geodataframe for geopandas
# we want a GDF with really just the census block IDs and the shapely geometry objects

gdf_blocks = gpd.GeoDataFrame(blocks, geometry="geometry", crs="EPSG:4326")[["GEOID20", "geometry"]]
gdf_blocks.head()

,GEOID20,geometry
0,60750101011001,"MULTIPOLYGON (((-122.42035 37.81151, -122.42 3..."
1,60750478011005,"MULTIPOLYGON (((-122.49504 37.77395, -122.4939..."
2,60750154011005,"MULTIPOLYGON (((-122.44936 37.78369, -122.4483..."
3,60750329012003,"MULTIPOLYGON (((-122.49001 37.74803, -122.4889..."
4,60750154012002,"MULTIPOLYGON (((-122.45681 37.78393, -122.4557..."


In [34]:
sales_tax = pd.read_csv("raw_data/sales-tax.csv")
sales_tax.head()

,quarter,area,amount
0,2025Q2,60750101011005 | 60750105001002 | 607501050010...,"118,140.44"
1,2025Q2,60750101011015 | 60750101012002 | 607501010120...,"29,328.64"
2,2025Q2,60750101011016 | 60750101012000 | 607501020210...,"61,111.45"
3,2025Q2,60750101011020 | 60750101021001 | 607501010210...,"41,524.44"
4,2025Q2,60750101012001 | 60750101012007 | 607501010120...,"43,155.05"


In [35]:
# convert sales tax to int at some point
sales_tax.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44042 entries, 0 to 44041
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   quarter  44042 non-null  object
 1   area     44042 non-null  object
 2   amount   44042 non-null  object
dtypes: object(3)
memory usage: 1.0+ MB


In [36]:
# we need a unique identifier for each year + quarter 

sales_tax["group_id"] = sales_tax.index
sales_tax.head()

,quarter,area,amount,group_id
0,2025Q2,60750101011005 | 60750105001002 | 607501050010...,"118,140.44",0
1,2025Q2,60750101011015 | 60750101012002 | 607501010120...,"29,328.64",1
2,2025Q2,60750101011016 | 60750101012000 | 607501020210...,"61,111.45",2
3,2025Q2,60750101011020 | 60750101021001 | 607501010210...,"41,524.44",3
4,2025Q2,60750101012001 | 60750101012007 | 607501010120...,"43,155.05",4


In [39]:
# clean up the area column

sales_tax["block_list"] = sales_tax["area"].str.replace(" ", "", regex = False)
sales_tax["block_list"] = sales_tax["block_list"].str.split("|")
sales_tax.head()

,quarter,area,amount,group_id,block_list
0,2025Q2,60750101011005 | 60750105001002 | 607501050010...,"118,140.44",0,"[60750101011005, 60750105001002, 6075010500102..."
1,2025Q2,60750101011015 | 60750101012002 | 607501010120...,"29,328.64",1,"[60750101011015, 60750101012002, 6075010101200..."
2,2025Q2,60750101011016 | 60750101012000 | 607501020210...,"61,111.45",2,"[60750101011016, 60750101012000, 6075010202100..."
3,2025Q2,60750101011020 | 60750101021001 | 607501010210...,"41,524.44",3,"[60750101011020, 60750101021001, 6075010102100..."
4,2025Q2,60750101012001 | 60750101012007 | 607501010120...,"43,155.05",4,"[60750101012001, 60750101012007, 6075010101200..."


In [42]:
# separate the list and rename for merge

sales_tax_long = sales_tax.explode("block_list", ignore_index=True)
sales_tax_long = sales_tax_long.rename(columns={"block_list": "GEOID20"})
sales_tax_long["GEOID20"] = sales_tax_long["GEOID20"].astype("string")
sales_tax_long.head()

,quarter,area,amount,group_id,GEOID20
0,2025Q2,60750101011005 | 60750105001002 | 607501050010...,"118,140.44",0,60750101011005
1,2025Q2,60750101011005 | 60750105001002 | 607501050010...,"118,140.44",0,60750105001002
2,2025Q2,60750101011005 | 60750105001002 | 607501050010...,"118,140.44",0,60750105001021
3,2025Q2,60750101011005 | 60750105001002 | 607501050010...,"118,140.44",0,60750101011000
4,2025Q2,60750101011005 | 60750105001002 | 607501050010...,"118,140.44",0,60750101011007


- we added a unique identifier to each year+ quarter
- we split the census blocks to be grouped into different rows
- we renamed the geometry merging column

### Join geometries

- merge sales tax data with shapely object geometry 
- convert to geodataframe 
- group the geometries together for each unique group_id
- retain the sales tax number

In [45]:
gdf_sales = sales_tax_long.merge(gdf_blocks, on = "GEOID20", how = 'left')

gdf_sales = gpd.GeoDataFrame(gdf_sales, geometry="geometry", crs="EPSG:4326")

gdf_sales.head()

,quarter,area,amount,group_id,GEOID20,geometry
0,2025Q2,60750101011005 | 60750105001002 | 607501050010...,"118,140.44",0,60750101011005,"MULTIPOLYGON (((-122.41584 37.80897, -122.4155..."
1,2025Q2,60750101011005 | 60750105001002 | 607501050010...,"118,140.44",0,60750105001002,"MULTIPOLYGON (((-122.4035 37.80509, -122.40234..."
2,2025Q2,60750101011005 | 60750105001002 | 607501050010...,"118,140.44",0,60750105001021,"MULTIPOLYGON (((-122.40499 37.80383, -122.4033..."
3,2025Q2,60750101011005 | 60750105001002 | 607501050010...,"118,140.44",0,60750101011000,"MULTIPOLYGON (((-122.42108 37.81289, -122.4201..."
4,2025Q2,60750101011005 | 60750105001002 | 607501050010...,"118,140.44",0,60750101011007,"MULTIPOLYGON (((-122.40905 37.80807, -122.4074..."


In [47]:
gdf_sales["geometry"].isna().sum()

0

In [49]:
# dissolve to get one reshaped polygon per row. 

gdf_groups = gdf_sales.dissolve(by="group_id", aggfunc="first").reset_index()

gdf_groups.head()

,group_id,geometry,quarter,area,amount,GEOID20
0,0,"MULTIPOLYGON (((-122.40316 37.8031, -122.40352...",2025Q2,60750101011005 | 60750105001002 | 607501050010...,"118,140.44",60750101011005
1,1,"MULTIPOLYGON (((-122.39848 37.80723, -122.3933...",2025Q2,60750101011015 | 60750101012002 | 607501010120...,"29,328.64",60750101011015
2,2,"MULTIPOLYGON (((-122.40499 37.80383, -122.4053...",2025Q2,60750101011016 | 60750101012000 | 607501020210...,"61,111.45",60750101011016
3,3,"MULTIPOLYGON (((-122.41322 37.80376, -122.4137...",2025Q2,60750101011020 | 60750101021001 | 607501010210...,"41,524.44",60750101011020
4,4,"MULTIPOLYGON (((-122.42347 37.80436, -122.4250...",2025Q2,60750101012001 | 60750101012007 | 607501010120...,"43,155.05",60750101012001


In [50]:
# sanity checks
len(gdf_groups)

44042

In [51]:
len(sales_tax)

44042

In [52]:
# export to geojson to use in Mapbox
gdf_groups = gdf_groups.to_crs(4326)
gdf_groups.to_file("data_with_geometry/sf_sales_tax_polygons.geojson", driver="GeoJSON")

In [53]:
# Also save a version with geometry for computation

gdf_groups_wkt = gdf_groups.copy()
gdf_groups_wkt["geometry"] = gdf_groups_wkt["geometry"].astype(str)

gdf_groups_wkt.to_csv("data_with_geometry/sf_sales_tax_groups_with_geometry.csv", index=False)

/var/folders/2_/nl4h85lj1jqcl64vgwgvqnh40000gn/T/ipykernel_45151/2111373250.py:4: UserWarning: Geometry column does not contain geometry.
  gdf_groups_wkt["geometry"] = gdf_groups_wkt["geometry"].astype(str)
